<a href="https://colab.research.google.com/github/raymondwcs/learning_spacy/blob/main/spaCy_Attribute_Ruler%2C_Entity_Ruler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
!pip install --quiet -U spacy
!python -m spacy download zh_core_web_lg

     |████████████████████████████████| 603.8 MB 7.9 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('zh_core_web_lg')


In [14]:
!git clone https://github.com/raymondwcs/learning_spacy/

fatal: destination path 'learning_spacy' already exists and is not an empty directory.


In [4]:
import spacy

sentences = [
  '一齊係國企成份股調整期間 大家一齊不問價掃貨 尤其是386 857 成份股佔指數比重只係5%左右 冇咩受新計法影響 挾死班歐美鬼佬 港股10月29000 遠必挾之 foshan fst_evil.gif',
  '恆指上29000 smile_22.gif  qq400 smile_22.gif ',
  'a股牛市開始😎',
  '我都唔知同佢講乜xyz123'
]

In [23]:
nlp = spacy.load('zh_core_web_lg')
nlp.tokenizer.initialize(pkuseg_model="./learning_spacy/spacy_pkuseg/models")
# nlp.tokenizer.pkuseg_update_user_dict([])

attribute_patterns = [
  { 
    "patterns" :[[{"TEXT": {"REGEX": ".*\.(gif|jpe?g|png)$"}}]],
    "attrs" : {"TAG": "STICKER", "POS":"ADJ"}
  }
]
attribute_ruler = nlp.get_pipe('attribute_ruler')
attribute_ruler.add_patterns(attribute_patterns)

entity_patterns = [
    {"label": "STOCK", "pattern": "386"},
    {"label": "STOCK", "pattern": "857"},
]

entity_ruler = nlp.add_pipe("entity_ruler", before='ner')
entity_ruler.add_patterns(entity_patterns)

In [37]:
def myTokenizer(sentence):
  doc = nlp(sentence)
  list = []
  oov = []
  for token in doc:
    print(token.text, token.tag_, token.pos_)
    if token.is_oov and token.text not in oov:  # out of dictionary
      oov.append(token.text)
    if token.tag_ == 'STICKER':
      if token.text == 'smile_22.gif':
        list.append('😡')
      elif token.text == 'fst_evil.gif':
        list.append('😈')
      elif token.text == 'smile_34.gif':
        list.append('😎')
    elif token.pos_ == 'NOUN' or token.pos_ == 'VERB' or token.pos_ == 'PROPN' and token.text not in list:
      list.append(token.text)

  for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)
    if ent.label_ == 'STOCK' and ent.text not in list:
      list.append(ent.text)

  print(list)
  print('OOV: {}\n'.format(oov))
  return(list)

In [38]:
sentence_tokens = []
for sentence in sentences:
  tokens = myTokenizer(sentence)
  sentence_tokens.append(tokens)

for s, k in zip(sentences,sentence_tokens):
  print(s)
  print(k)
  print()

一齊 AD ADV
係 NN NOUN
國企 NR PROPN
成份股 NN NOUN
調整 VV VERB
期間 LC PART
大家 PN PRON
一齊 AD ADV
不 AD ADV
問價 VV VERB
掃貨 VV VERB
尤其 AD ADV
是 VC VERB
386 CD NUM
857 CD NUM
成份 NN NOUN
股佔 NN NOUN
指數 NN NOUN
比重 NN NOUN
只 AD ADV
係 VV VERB
5% CD NUM
左右 LC PART
冇咩 VV VERB
受 VV VERB
新 AD ADV
計法 VV VERB
影響 NN NOUN
挾 VV VERB
死班 NR PROPN
歐美 NR PROPN
鬼佬 NN NOUN
港股 NN NOUN
10月 NT NOUN
29000 CD NUM
遠必 VV VERB
挾 VV VERB
之 DEC PART
foshan NN NOUN
fst_evil.gif STICKER ADJ
一齊 0 2 CARDINAL
386 26 29 STOCK
857 30 33 STOCK
5% 44 46 CARDINAL
['係', '國企', '成份股', '調整', '問價', '掃貨', '是', '成份', '股佔', '指數', '比重', '係', '冇咩', '受', '計法', '影響', '挾', '死班', '歐美', '鬼佬', '港股', '10月', '遠必', '挾', 'foshan', '😈', '386', '857']
OOV: ['一齊', '問價', '股佔', '冇咩', '計法', '死班', '遠必', 'foshan', 'fst_evil.gif']

恆指 NN NOUN
上 VV VERB
29000 CD NUM
smile_22.gif STICKER ADJ
  _SP SPACE
qq NN NOUN
400 OD NUM
smile_22.gif STICKER ADJ
29000 smile_22.gif 3 21 QUANTITY
400 smile_22.gif 25 41 QUANTITY
['恆指', '上', '😡', 'qq', '😡']
OOV: ['smile_22.gif', ' ']

a